# Training a neural net to classify personal attacks in Wikipedia comments

In [1]:
import keras
import os
import pandas as pd
import urllib


from keras.layers.core import Dense, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


## Load and pre-process the data.
### This is the exact preprocessing method used by Wulczyn, et al, the curators of the data.

In [ ]:
# Download annotated comments and annotations. 
# If you're Tracy, Courtney, or Amandalynne, don't run this step 
# because you already have the data! If you aren't us, you will 
# probably need to do this step. 
# It will take a while. 
ANNOTATED_COMMENTS_URL = 'https://ndownloader.figshare.com/files/7038044' 
ANNOTATIONS_URL = 'https://ndownloader.figshare.com/files/7383751' 


def download_file(url, fname):
    urllib.request.urlretrieve(url, fname)

                
download_file(ANNOTATED_COMMENTS_URL, 'attack_annotated_comments.tsv')
download_file(ANNOTATIONS_URL, 'attack_annotations.tsv')

In [2]:
# Read the data into a Pandas dataframe.
comments = pd.read_csv('attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('attack_annotations.tsv',  sep = '\t')

# Label a comment as an attack if over half of annotators did so.
# We can tinker with this threshold later.
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5

# Join labels and comments
comments['attack'] = labels

# Preprocess the data -- remove newlines, tabs, quotes
# Something to consider: remove Wikipedia style markup (::'s and =='s)
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("`", " "))

In [3]:
# Take a look at the comments
# Look: they've already split the data into train / dev / test :) 
comments.head(5)

,comment,year,logged_in,ns,sample,split,attack
rev_id,,,,,,,
37675,- This is not creative . Those are the di...,2002,False,article,random,train,False
44816,:: the term standard model is itself le...,2002,False,article,random,train,False
49851,"True or false, the situation as of March 200...",2002,False,article,random,train,False
89320,"Next, maybe you could work on being less cond...",2002,True,article,random,dev,False
93890,This page will need disambiguation.,2002,True,article,random,train,False


In [4]:
# Grab the training, dev, and test data (3:1:1 split)
train_data = comments.loc[comments['split'] == 'train']
dev_data = comments.loc[comments['split'] == 'dev']
test_data = comments.loc[comments['split'] == 'test']

In [5]:
# The list of gold-standard labels for the data
train_labels = train_data["attack"].tolist()
dev_labels = dev_data["attack"].tolist()
test_labels = test_data["attack"].tolist()

In [5]:
"""
## THIS WAS FOR COUNTING UP THE PROPORTIONS OF ATTACKS IN EACH SPLIT
## IGNORE!
dev_data = comments.loc[comments['split'] == 'dev']
test_data = comments.loc[comments['split'] == 'test']
print(train_data.shape)
print(dev_data.shape)
print(test_data.shape)


train_attacks = train_data.loc[train_data['attack'] == True]
dev_attacks = dev_data.loc[dev_data['attack'] == True]
test_attacks = test_data.loc[test_data['attack'] == True]

print(train_attacks.shape)
print(dev_attacks.shape)
print(test_attacks.shape)
"""

"\n## THIS WAS FOR COUNTING UP THE PROPORTIONS OF ATTACKS IN EACH SPLIT\n\ndev_data = comments.loc[comments['split'] == 'dev']\ntest_data = comments.loc[comments['split'] == 'test']\nprint(train_data.shape)\nprint(dev_data.shape)\nprint(test_data.shape)\n\n\ntrain_attacks = train_data.loc[train_data['attack'] == True]\ndev_attacks = dev_data.loc[dev_data['attack'] == True]\ntest_attacks = test_data.loc[test_data['attack'] == True]\n\nprint(train_attacks.shape)\nprint(dev_attacks.shape)\nprint(test_attacks.shape)\n"

In [6]:
train_data.head(5)

,comment,year,logged_in,ns,sample,split,attack
rev_id,,,,,,,
37675,- This is not creative . Those are the di...,2002,False,article,random,train,False
44816,:: the term standard model is itself le...,2002,False,article,random,train,False
49851,"True or false, the situation as of March 200...",2002,False,article,random,train,False
93890,This page will need disambiguation.,2002,True,article,random,train,False
102817,- Important note for all sysops: There is a ...,2002,True,user,random,train,False


In [7]:
# Put all the training data (comments) into a list
train_texts = train_data["comment"].tolist()
# If you want to see a snippet of it for a sanity check:
# train_texts[:3]

In [8]:
len(train_texts)

69526

In [9]:
# A set of character unigrams.
# This number may come in handy later
char_unigrams = set(''.join(train_texts))

# Note: there are 1557 unique characters.

In [10]:
len(char_unigrams)

1557

In [11]:
# This function makes the char ngrams we want. 
# Default to 1-5, as in the paper
def char_ngram_generator(text, n1=1, n2=5):
    z = []
    text2 = '*'+text+'*'
    for k in range(n1,n2):
        z.append([text2[i:i+k] for i in range(len(text2)-k+1)])
    z = [ngram for ngrams in z for ngram in ngrams]
    z.append(text)
    return set(z)

# Fiddling around with keras Tokenizer to make the character embeddings. Work in progress.

In [12]:
# The char-level tokenizer will only produce character unigrams.
# Figuring out the 2..5 grams will come another day.
# We may want to set a threshold for frequency, e.g. only care about
# top 100 most frequent chars / char sequences. 
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [13]:
# Fit it to the training data.
tokenizer.fit_on_texts(train_texts)

In [14]:
# Take a look at the character counts for fun
tokenizer.word_counts

{' ': 5351570,
 '-': 51321,
 'T': 113251,
 'h': 943891,
 'i': 1572357,
 's': 1326059,
 'n': 1417949,
 'o': 1659847,
 't': 1889196,
 'c': 612939,
 'r': 1188214,
 'e': 2486567,
 'a': 1696248,
 'v': 214828,
 '.': 294108,
 'd': 764831,
 'y': 454043,
 'f': 392185,
 'm': 487940,
 'u': 657833,
 'p': 426631,
 'l': 856363,
 'I': 189734,
 "'": 98998,
 ',': 209650,
 'g': 427725,
 'w': 341809,
 'b': 310011,
 'q': 19512,
 'Y': 40097,
 'j': 31726,
 'k': 223359,
 '(': 31615,
 ')': 34335,
 'R': 55088,
 'A': 114695,
 'B': 42454,
 'x': 39931,
 ':': 105879,
 'M': 48770,
 'D': 44483,
 'U': 48932,
 'S': 88004,
 '1': 35413,
 '9': 15515,
 '5': 12718,
 '0': 41861,
 '2': 28258,
 'G': 40618,
 'W': 67605,
 '?': 34090,
 'C': 68171,
 'N': 63965,
 'F': 44156,
 'P': 55064,
 'O': 71824,
 'V': 14918,
 'K': 29262,
 'z': 19348,
 'L': 46460,
 'H': 62782,
 '[': 3258,
 ']': 3661,
 'J': 18911,
 '6': 10077,
 '!': 66567,
 ';': 11675,
 'E': 76531,
 '<': 203,
 '>': 721,
 '/': 27683,
 'ö': 139,
 '4': 11216,
 '3': 14261,
 '_': 81

In [21]:
def texts_to_matrix(data, tokenizer):
    """Given a section of the data, return a matrix representing comments"""
    texts = data["comment"].tolist()
    matrix = tokenizer.texts_to_matrix(texts)
    return matrix

In [22]:
# Transform each comment in the training data to arrays of equal length
# I should really make this tokenization step into a function! To-do for AP.
train_matrix = texts_to_matrix(train_data, tokenizer)

In [23]:
# Dimensions of our training matrix
train_matrix.shape[0], train_matrix.shape[1]

(69526, 1558)

In [24]:
len(train_matrix[1])

1558

In [18]:
# Make a model.
model = Sequential()
# Add embedding layer.
# Recall from earlier that char unigram vocab size is 1557.
# Train_matrix shape is 69526 x 1558, use that as input dimension.
# Output dimension of.. um..
# There's probably something wrong here so fix later.
model.add(Embedding(1558, 10, input_length=train_matrix.shape[1]))
model.add(Flatten())
# Commented out for now bc... the example I was following only had one Dense layer
#model.add(Dense(2, input_dim=len(char_unigrams)))
model.add(Dense(1, activation='sigmoid'))

In [19]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
# Validate on the training data to start out.
# Somehow manages to get accuracies, lol.
model.fit(train_matrix, train_labels,
          batch_size=100,
          epochs=2,
          validation_data=(train_matrix, train_labels))

Train on 69526 samples, validate on 69526 samples
Epoch 1/2
69526/69526 [==============================] - 10s - loss: 0.3145 - acc: 0.8882 - val_loss: 0.2887 - val_acc: 0.8936

In [25]:
# How about on the dev data?
dev_matrix = texts_to_matrix(dev_data, tokenizer)
model.fit(train_matrix, train_labels,
          batch_size=100,
          epochs=2,
          validation_data=(dev_matrix, dev_labels))

Train on 69526 samples, validate on 23160 samples
Epoch 1/2
69526/69526 [==============================] - 10s - loss: 0.2874 - acc: 0.8949 - val_loss: 0.2870 - val_acc: 0.8920